In [ ]:
import numpy as np
import cv2
input_image = cv2.imread('foreground.jpg')
background_array = np.zeros((1,65),np.float64)
foreground_array = np.zeros((1,65),np.float64)

def get_merge_mask(foreground_mask, background_mask, input_image):
    get_mask = foreground_mask[:,:,1:2]/255 + background_mask[:,:,2:3]/255
    if np.max(np.max(np.max(get_mask)))>1:
        return False, input_image
    get_mask = get_mask.astype('uint8')
    return True, foreground_mask + background_mask + input_image*(1-get_mask)

draw_lines = False
method = True
x_value,y_value = -1,-1
r = 8

def interactive_draw(event,x_axis,y_axis,flags,param):
    if event == cv2.EVENT_LBUTTONDOWN:
        draw_lines = True
        x_value,y_value = x_axis,y_axis
    if event == cv2.EVENT_RBUTTONUP:
        method = not method

    elif event == cv2.EVENT_MOUSEMOVE:
        if draw_lines is True:
            if method is True:
                cv2.circle(foreground_mask,(x_axis,y_axis),r,(0,255,0),-1)
            else:
                cv2.circle(background_mask,(x_axis,y_axis),r,(0,0,255),-1)

    elif event == cv2.EVENT_LBUTTONUP:
        draw_lines = False
        if method is True:
            cv2.circle(foreground_mask,(x_axis,y_axis),r,(0,255,0),-1)
        else:
            cv2.circle(background_mask,(x_axis,y_axis),r,(0,0,255),-1)
                         
foreground_mask = np.zeros_like(input_image)
background_mask = np.zeros_like(input_image)
cv2.namedWindow('grabcut algorithm')
cv2.setMouseCallback('grabcut',interactive_draw)
while True:
    source, apply_mask = get_merge_mask(foreground_mask, background_mask, input_image)
    if not source:
        foreground_mask = np.zeros_like(input_image)
        background_mask = np.zeros_like(input_image)
    cv2.imshow('image',apply_mask)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('m'):
        method = not method
        if method:
            print('front')
        else:
            print('back')
    elif key == 13:
        print('cropping')
        output_mask = np.zeros(input_image.shape[:2],np.uint8) + 2
        output_mask[foreground_mask[:,:,1] == 255] = 1
        output_mask[background_mask[:,:,2] == 255] = 0
        output_mask, background_array,foreground_array = cv2.grabCut(input_image,output_mask,None,background_array,foreground_array,iteration,cv2.GC_INIT_WITH_MASK)        
        output_mask = np.where((output_mask==2)|(output_mask==0),0,1).astype('uint8')
        final_out = input_image*output_mask[:,:,np.newaxis]
        cv2.imshow('final_out',final_out)
        cv2.imwrite('output.jpg',final_out)
    elif key == 27:
        break
    cv2.destroyAllWindows()